In [1]:
# import RPi.GPIO as GPIO
import time
from dahuffman import HuffmanCodec
import json

In [2]:
# GPIO.setmode(GPIO.BOARD)
data_pin = 12
# GPIO.setup(data_pin, GPIO.OUT, initial=0)

filename = 'message.txt'
f = open(filename, 'r')
msg = f.read()
print(msg, len(msg))
time_period = 0.005  # seconds
len_msg = len(msg)
max_payload = 4 # bytes
max_bytes_processing = 40 # bytes
codec = HuffmanCodec.from_data(msg) 
codec.print_code_table()
# codec.load = codec.get_code_table
# codec_bytes = json.dumps(codec).encode('utf-8')
# codec_receiver = json.loads(codec_bytes.decode('utf-8'))

hi, this is a demo message for transmission. 44
Bits Code   Value Symbol
   5 00000      0 'f'
   5 00001      1 'g'
   4 0001       1 'h'
   4 0010       2 'n'
   4 0011       3 'r'
   3 010        2 'i'
   4 0110       6 't'
   4 0111       7 'a'
   4 1000       8 'e'
   4 1001       9 'm'
   3 101        5 ' '
   4 1100      12 'o'
   6 110100    52 _EOF
   6 110101    53 ','
   6 110110    54 '.'
   6 110111    55 'd'
   3 111        7 's'


In [3]:
def print_stats(duration):
    print('Total bits send: ', len_msg * 8)
    print('Total time taken(s): ', duration)
    print('Datarate(bps): ', len_msg * 8 / (duration))

def add_header(payload,type_payload,len_payload):
    '''
    type: 0 for table, 1 for data
    len: 4 bits representing maximum of 15 bytes
    '''
    
    header = [type_payload,len_payload]

    return bytes(header) + payload


def fragment_bytes(msg_bytes,index):
    '''
    returns the fragment of bytes that can be send in one processing
    ex - total msg size is 100 bytes, then only 40 can be processed at once which 
    will be send in packets of 4 bytes
    '''
    if(len_msg - index>max_bytes_processing):
        return msg_bytes[index:index+max_bytes_processing]
    else:
        return msg_bytes[index:]

def prepare_packet(fragment,type_payload, index):
    '''
    returns the packet with header and payload
    '''
    if(len(fragment) - index >= max_payload):
        len_payload = max_payload
    else:
        len_payload = len(fragment) - index

    return add_header(fragment[index:index+max_payload],type_payload,len_payload)

def transmit_byte(byte):
    # bit_stream = byte
    # print(bit_stream)
    i = 0
    while (i < 8):
        print(1 if byte & (1 << i) else 0, end = '')
        # GPIO.output(data_pin, byte & (1 << i))
        time.sleep(time_period)
        i += 1
    # print('\n')

def run():
    '''
    protocol for sending message:
    1. send the encoding table
    2. send data in packets of max_payload bytes
    '''

    # codec = HuffmanCodec.from_data(msg)
    encoded_msg = codec.encode(msg)
    len_encoded_msg = len(encoded_msg)
    print(encoded_msg)
    
    for i in range(1):
        # send the encoding table
        # print('sending encoding table')
        
        index = 0
        start = time.time()
        print('Starting transmission\n')
        while(index < len_msg):

            fragment = fragment_bytes(encoded_msg,index)
            index += max_bytes_processing
            i = 0
            while(i < len(fragment)):
                print('sending packet')
                packet = prepare_packet(fragment,1,i)
                print(packet)
                for byte in packet:
                    transmit_byte(byte)

                i += max_payload

        end = time.time()
        print_stats(end - start)


In [4]:
run()

b'\x15\xada^\xaf^\xef\x13\x96c\xf7\x0cPa\xd67/+\xf5\x85\xb6'
Starting transmission

sending packet
b'\x01\x04\x15\xada^'
100000000010000010101000101101011000011001111010sending packet
b'\x01\x04\xaf^\xef\x13'
100000000010000011110101011110101111011111001000sending packet
b'\x01\x04\x96c\xf7\x0c'
100000000010000001101001110001101110111100110000sending packet
b'\x01\x04Pa\xd67'
100000000010000000001010100001100110101111101100sending packet
b'\x01\x04/+\xf5\x85'
100000000010000011110100110101001010111110100001sending packet
b'\x01\x01\xb6'
100000001000000001101101Total bits send:  352
Total time taken(s):  4.11861515045166
Datarate(bps):  85.46562063741221


In [40]:
def receive_byte():
    bit_stream = 0
    i = 0
    while (i < 8):
        bit_stream |= GPIO.input(data_pin) << i
        time.sleep(time_period)
        i += 1
    return bit_stream



In [7]:
a = [1,2]
b = [3,4]
bytes(a) + bytes(b)

b'\x01\x02\x03\x04'

In [44]:
ord('(')

40